# **Neuroimaging 2025/2026**

**Lab Structural MRI**

Tutors: Beatriz Vale & Vânia Miguel

Created by: Marta Xavier & Neil Mehta


## **Objectives**
```
Implement main workflow to process structural Magnetic Resonance Imaging (MRI) images:

* Setting up MRI analysis packages;
* Perform brain extraction using different analysis packages (ANTS, AFNI, FreeSurfer, FSL);
* Perform image segmentation into the main brain tissue classes;
* Perform image registration, by applying linear and nonlinear transformations.  

```

# **I. Set up the environment**

In [ ]:
# Set up Neurodesk
%%capture
import os
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    os.environ["LD_PRELOAD"] = "";
    os.environ["APPTAINER_BINDPATH"] = "/content"
    os.environ["MPLCONFIGDIR"] = "/content/matplotlib-mpldir"
    os.environ["LMOD_CMD"] = "/usr/share/lmod/lmod/libexec/lmod"

    !curl -J -O https://raw.githubusercontent.com/neurodesk/neurocommand/main/googlecolab_setup.sh
    !chmod +x googlecolab_setup.sh
    !./googlecolab_setup.sh

    os.environ["MODULEPATH"] = ':'.join(map(str, list(map(lambda x: os.path.join(os.path.abspath('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/'), x),os.listdir('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/')))))

In [ ]:

# Download analysis packages
import lmod

# Set up AFNI
await lmod.load('afni/24.1.02')

# Set up FreeSurfer
await lmod.load('freesurfer/7.3.2')

# Set up FSL
await lmod.load('fsl/6.0.7.4')

# **II. Download example data**

Example data from FMRIB Software Library (FSL): [(link)](https://open.win.ox.ac.uk/pages/fslcourse/website/downloads.html)

## 1. Download dataset

In [ ]:
# Download T1-weighted (T1w) and T2-weighted (T2w) MRI structural data from FSL
!wget https://www.fmrib.ox.ac.uk/primers/intro_primer/ExBox17/ExBox17.zip
!unzip ExBox17.zip
!rm ExBox17.zip

# Permissions
!chmod -R 777 *

In [ ]:
# Change working directory
path = "ExBox17"
os.chdir(path)

In [ ]:
# Copy standard structural images from MNI to current directory
!cp $FSLDIR/data/standard/MNI152_T1_2mm.nii.gz standard.nii.gz
!cp $FSLDIR/data/standard/MNI152_T1_2mm_brain.nii.gz standard_brain.nii.gz
!cp $FSLDIR/etc/flirtsch/T1_2_MNI152_2mm.cnf .

## 2. Dataset contents



You can check the contents of the `ExBox17` folder in the files tab of this notebook, or by listing the contents of the working directory, using the SHELL command `!ls`

In [ ]:
# List the contents of the current directory
!ls

```
Contents:
* T1-weighted (T1-w) structural scan: T1
* T1-w structural scan, skull-stripped: T1_brain
* T2-weighted (T2-w) structural scan: T2
* T2-w structural scan, skull-stripped: T2_brain
* Standard MNI template of structural scan: standard
* Standard MNI template of structural scan, skull-stripped: standard_brain

```

### Check dataset images

In [ ]:
# Import visualization packages
from ipyniivue import AnyNiivue
from IPython.display import display, Markdown

In [ ]:
# Check T1w image info
!fslinfo T1

**Relevant informations:**

- Number of voxels: `dim1`, `dim2`, `dim3`.
- Voxel size: `pixdim1`, `pixdim2`, `pixdim3`.

In [ ]:
# Visualize T1w image
display(Markdown("### T1w structural image"))

nv_T1 = AnyNiivue()
nv_T1.load_volumes([{"path": "T1.nii.gz"}])
nv_T1

**Images, from left to right:**

- Axial slices: A (anterior) on top; P (posterior) on the bottom; L (left) on the left, R (right) on the right.
- Coronal slices: S (superior) on top, I (inferior) on the bottom; L (left) on the left, R (right) on the right.
- Sagittal slices: S (superior) on top, I (inferior) on the bottom; P (posterior) on the left, A (anterior) on the right.
- 3D rendition of the brain.

In [ ]:
# Check T2w image info
!fslinfo T2

In [ ]:
# Visualize T2w image
display(Markdown("### T2w structural image"))

nv_T1 = AnyNiivue()
nv_T1.load_volumes([{"path": "T2.nii.gz"}])
nv_T1

T1 and T2 are, respectively, the longitudinal and transverse relaxation time. Different tissues have different T1 and T2 values:
  - T1 (WM) > T1 (GM): white matter is brighter in T1w images.
  - T2 (WM) < T2 (GM): grey matter is brighter in T2w images.

# **III. Brain extraction**

Adapted from [(link)](https://www.neurodesk.org/example-notebooks/structural_imaging/brain_extraction_different_tools.html#)

MRI studies focus on brain tissue, so typically the first processing step is to remove the skull and non-brain areas from the image.

This is usually called _brain extraction_ or _skull-stripping_.

In this example we will use the following MRI analysis packages and algorithms to skull-strip the anatomical image:

*   Analysis of Functional NeuroImages (AFNI): 3dSkullStrip [(link)](https://afni.nimh.nih.gov/pub/dist/doc/program_help/3dSkullStrip.html)
*   FreeSurfer: SynthStrip [(link)](https://surfer.nmr.mgh.harvard.edu/docs/synthstrip/)
*   FSL (FMRIB Software Library): BET  [(link)](https://open.win.ox.ac.uk/pages/fslcourse/practicals/intro2/index.html)

## 1. AFNI: 3dSkullStrip

AFNI is a suite of programs designed to analyze fMRI data.
3dSkullStrip can be used for brain extraction. [(link)](https://afni.nimh.nih.gov/pub/dist/doc/program_help/3dSkullStrip.html)

In [ ]:
# Usage
!3dSkullStrip -help

In [ ]:
# Run
!3dSkullStrip -input T1.nii.gz -prefix T1_brain_AFNI.nii.gz -push_to_edge

## 2. Freesurfer:  SynthStrip

In [ ]:
# Usage
!mri_synthstrip --help

In [ ]:
# Run
!mri_synthstrip -i T1.nii.gz -o T1_brain_freesurfer.nii.gz -m synth_mask.nii.gz

FreeSurfer is a software package that enables you to analyze structural MRI images. You can use FreeSurfer to quantify the amount of grey matter and white matter in specific regions of the brain. You will also be able to calculate measurements such as the thickness, curvature, and volume of the different tissue types, and be able to correlate these with covariates; or, you can contrast these structural measurements between groups.

FreeSurfer’s SynthStrip is a skull-stripping tool that extracts brain voxels from a landscape of image types, ranging across imaging modalities, resolutions, and subject populations. It leverages a deep learning strategy to synthesize arbitrary training images from segmentation maps, yielding a robust model agnostic to acquisition specifics. [(link)](https://surfer.nmr.mgh.harvard.edu/docs/synthstrip/)

## 5. FSL: BET

FSL is a library of analysis tools for fMRI, MRI and diffusion brain imaging data.

FSL has a tool to skull-strip an anatomical image called BET (Brain Extraction Tool). [(link)](https://open.win.ox.ac.uk/pages/fslcourse/practicals/intro2/index.html), [(link)](https://fsl.fmrib.ox.ac.uk/fsl/docs/#/structural/bet)

In [ ]:
# Usage
!bet

In [ ]:
# Run BET
!bet T1 T1_brain -f 0.5 # define intensity threshold = 0.5

In [ ]:
# Run BET with varying intensity threshold
!bet T1 T1_brain_f02 -f 0.2 # define intensity threshold = 0.2

In [ ]:
# Run BET and create a binary brain mask
!bet T1 T1_brain.nii.gz -f 0.5 -m

## 6. Visualize Results

We can compare the original T1w MRI image with the skull-stripped results from the different analysis softwares used.

In [ ]:
# Visualize original T1 image (before brain extraction)
display(Markdown("### T1w structural image"))

nv_T1 = AnyNiivue()
nv_T1.load_volumes([{"path": "T1.nii.gz"}])
nv_T1

In [ ]:
# Visualize AFNI results
display(Markdown("### AFNI 3dSkullStrip"))

nv_AFNI = AnyNiivue()
nv_AFNI.load_volumes([{"path": "T1_brain_AFNI.nii.gz"}])
nv_AFNI

In [ ]:
# Visualize FreeSurfer results
display(Markdown("### FreeSurfer SynthStrip"))

nv_FreeSurfer = AnyNiivue()
nv_FreeSurfer.load_volumes([{"path": "T1_brain_freesurfer.nii.gz"}])
nv_FreeSurfer

In [ ]:
# Visualize FSL's BET results
# Threshold = 0.5 (default)
display(Markdown("### FSL BET (Thr=0.5)"))

nv_FSL = AnyNiivue()
nv_FSL.load_volumes([{"path": "T1_brain.nii.gz"}])
nv_FSL

In [ ]:
# Visualize FSL's BET results
# Threshold = 0.2
display(Markdown("### FSL BET (Thr=0.2)"))

nv_FSL = AnyNiivue()
nv_FSL.load_volumes([{"path": "T1_brain_f02.nii.gz"}])
nv_FSL

In [ ]:
# Visualize FSL's BET results
# Binary brain mask
display(Markdown("### FSL BET (Thr=0.5), Brain Mask"))

nv_FSL = AnyNiivue()
nv_FSL.load_volumes([{"path": "T1_brain_mask.nii.gz"}])
nv_FSL

# **IV. Tissue segmentation**

Adapted from [(link)](https://open.win.ox.ac.uk/pages/fslcourse/practicals/seg_struc/index.html)



We can use FSL's tools FIRST and FAST to perform tissue segmentation.

## 1. FAST: tissue type segmentation and bias-field correction

FSL's tool FAST can be used to perform bias field correction, as well as tissue type segmentation, on the skull stripped T1w structural image. [(link)](https://fsl.fmrib.ox.ac.uk/fsl/docs/#/structural/fast)

_Tissue type segmentation_: segment the structural image into grey matter, white matter and  CSF.

_Bias-field correction_: correct the artifacts resultant of uneven intisities due to RF or B1 inhomogeneities.


### **Run FAST**

In [ ]:
# Usage
!fast

In [ ]:
# Run fast (with bias field correction)
!fast -b -B T1_brain

### **Check Outputs**

*  Bias field corrected image: `T1_brain_restore`
*  Bias field: `T1_brain_bias`
*  CSF image: `T1_brain_pve_0`
*  Grey matter image: `T1_brain_pve_1`
*  White matter image: `T1_brain_pve_2`

### **Visualize Results**

In [ ]:
# Visualize FSL's FAST results
# Bias field
display(Markdown("### FSL FAST - Bias field"))

nv_FAST_bias = AnyNiivue()
nv_FAST_bias.load_volumes([{"path": "T1_brain_bias.nii.gz"}])
nv_FAST_bias

In [ ]:
# Visualize FSL's FAST results
# Bias field corrected T1w structural image
display(Markdown("### FSL FAST - Bias field corrected T1w image"))

nv_FAST_restore = AnyNiivue()
nv_FAST_restore.load_volumes([{"path": "T1_brain_restore.nii.gz"}])
nv_FAST_restore

In [ ]:
# Visualize FSL's FAST results
# CSF
display(Markdown("### FSL FAST - CSF"))

nv_FAST_CSF = AnyNiivue()
nv_FAST_CSF.load_volumes([{"path": "T1_brain_pve_0.nii.gz"}])
nv_FAST_CSF

In [ ]:
# Visualize FSL's FAST results
# Grey matter
display(Markdown("### FSL FAST - Grey matter"))

nv_FAST_GM = AnyNiivue()
nv_FAST_GM.load_volumes([{"path": "T1_brain_pve_1.nii.gz"}])
nv_FAST_GM

In [ ]:
# Visualize FSL's FAST results
# White Matter
display(Markdown("### FSL FAST - White matter"))

nv_FAST_WM = AnyNiivue()
nv_FAST_WM.load_volumes([{"path": "T1_brain_pve_2.nii.gz"}])
nv_FAST_WM

In [ ]:
# Visualize FSL's FAST results
# CSF, grey matter, white matter (overlayed)
display(Markdown("### FSL FAST - CSF (red), GM (blue), WM (white)"))

nv_FAST_all = AnyNiivue()
volumes = [{"path": "T1_brain_pve_2.nii.gz", "colormap": "white"},
           {"path": "T1_brain_pve_0.nii.gz", "colormap": "red"},
           {"path": "T1_brain_pve_1.nii.gz", "colormap": "blue"}]
nv_FAST_all.load_volumes(volumes)
nv_FAST_all

## 2. FIRST: segmentation of sub-cortical structures



FSL's tool FIRST can be used to perform segmentation of the skull stripped T1w structural image into several sub-cortical structures. [(link)](https://fsl.fmrib.ox.ac.uk/fsl/docs/#/structural/first?id=segmentation-with-run_first_all)


### **Run FIRST**

In [ ]:
# Usage
!run_first_all

In [ ]:
# Perform segmentation of sub-cortical structures
!run_first_all -i T1_brain -b -o T1_brain_segmented

### **Check Outputs**

* `T1_all_fast_firstseg`: This is a single image showing the segmented output for all structures. The image is produced by filling the estimated surface meshes and then running a step to ensure that there is no overlap between structures. The output uses the labels listed above (and a standard colour table is built into FSleyes). If another boundary correction method is specified, the name fast in this filename will change to reflect the boundary correction that is used. Note that if only one structure is specified then this file will be called T1-struct_corr.nii.gz instead (e.g. sub001-L_Caud_corr.nii.gz).

* `T1_all_fast_origsegs`: This is a 4D image containing the individual structure segmentations, converted directly from the native mesh representation and without any boundary correction. For each structure there is an individual 3D image where the interior is labeled according to the CMA standard labels while the boundary uses these label values plus 100. Note that if only one structure is specified then this file will be called T1-struct_first.nii.gz instead (e.g. sub001-L_Caud_first.nii.gz).

Example labels:
```
10 Left Thalamus (L_Thal); 11 Left Caudate (L_Caud); 12 Left Putamen (L_Puta); 13 Left Pallidum (L_Pall); 16 Brain Stem (BrStem); 17 Left Hippocampus (L_Hipp); 18 Left Amygdala (L_Amyg); 26 Left Accumbens (L_Accu); 49 Right Thalamus (R_Thal); 50 Right Caudate (R_Caud); 51 Right Putamen (R_Puta); 52 Right Pallidum (R_Pall); 53 Right Hippocampus (R_Hipp); 54 Right Amygdala (R_Amyg); 58 Right Accumbens (R_Accu)
```

### **Visualize Results**

In [ ]:
# Visualize FSL results
# FIRST
display(Markdown("### FSL FIRST - Segmentation"))

nv_FSL = AnyNiivue()
nv_FSL.load_volumes([{"path": "T1_brain_segmented_all_fast_firstseg.nii.gz"}])
nv_FSL


In [ ]:
!fslstats T1_brain_segmented_all_fast_firstseg -R

# **V. Registration**

Adapted from [(link)](https://www.fmrib.ox.ac.uk/primers/intro_primer/ExBox17/IntroBox17.html)



We can use FSL's tools FLIRT and FNIRT to perform linear and non-linear registration of MRI images.

## 1. FLIRT: within-subject registration




We can use linear registration, using FSL's FLIRT, for within-subject registration. [(link)](https://web.mit.edu/fsl_v5.0.10/fsl/doc/wiki/FLIRT(2f)UserGuide.html)

For example, to register a subject's T2-weighted image to the T1-weighted image, or to register a subject's functional image to the T1-weighted image.

In [ ]:
# Usage
!flirt

In [ ]:
# Perform brain extraction/skull stripping on T2w structural image
!bet T2 T2_brain

In [ ]:
# Visualize results (before registration)
display(Markdown("### T2w (blue) and T1w (red) - before registration"))

nv_FSL = AnyNiivue()
volumes =  [{"path": "T2_brain.nii.gz", "colormap": "blue"},
           {"path": "T1_brain.nii.gz", "colormap": "red"}]
nv_FSL.load_volumes(volumes)
nv_FSL

In [ ]:
# Use a linear transformation with 6 DOF to register the subject's T2w to T1w structural images
# Cost function: least squares
!flirt -in T2_brain.nii.gz -ref T1_brain.nii.gz -dof 6 -cost leastsq -omat T2toT1_LS.mat -out T2toT1_LS

# Cost function: mutual information
!flirt -in T2_brain -ref T1_brain.nii.gz -dof 6 -cost mutualinfo -omat T2toT1_MI.mat -out T2toT1_MI

In [ ]:
# Visualize results (after registration, LS)
display(Markdown("### T2w (blue) and T1w (red) - after registration (LS)"))

nv_FLIRT_LS = AnyNiivue()
volumes =  [{"path": "T2toT1_LS.nii.gz", "colormap": "blue"},
           {"path": "T1_brain.nii.gz", "colormap": "red"}]
nv_FLIRT_LS.load_volumes(volumes)
nv_FLIRT_LS

In [ ]:
# Visualize results (after registration, MI)
display(Markdown("### T2w (blue) and T1w (red) - after registration (MI)"))

nv_FLIRT_MI = AnyNiivue()
volumes =  [{"path": "T2toT1_MI.nii.gz", "colormap": "blue"},
           {"path": "T1_brain.nii.gz", "colormap": "red"}]
nv_FLIRT_MI.load_volumes(volumes)
nv_FLIRT_MI

## 2. FNIRT: between-subject registration

We can use non-linear registration, using FSL's FNIRT, for registration to standard space.

In this case, we will register a subject’s T1-weighted image to the T1-weighted image in the MNI standard space (MNI152_T1).

In [ ]:
# Usage
!fnirt

In [ ]:
# Visualize results (before registration)
display(Markdown("### Standard (blue) and T1w (red) - before registration"))

nv_FSL = AnyNiivue()
volumes =  [{"path": "standard_brain.nii.gz", "colormap": "blue"},
           {"path": "T1_brain.nii.gz", "colormap": "red"}]
nv_FSL.load_volumes(volumes)
nv_FSL



In [ ]:
# It's common to use FLIRT to obtain an initial approximation of the transformation, which we can later input in FNIRT
# Use an affine linear transformation with 12 DOF
!flirt -in T1_brain.nii.gz -ref standard_brain -dof 12 -out T1toMNIlin -omat T1toMNIlin.mat

# Provide the affine transformation matrix estimated in the step above to initialize the non-linear registration
!fnirt --in=T1_brain.nii.gz --aff=T1toMNIlin.mat --config=T1_2_MNI152_2mm.cnf --iout=T1toMNInonlin --cout=T1toMNI_coef --fout=T1toMNI_warp

In [ ]:
# Visualize results (after registration)
display(Markdown("### Standard MNI (blue) and T1w (red) - after registration"))

nv_FNIRT = AnyNiivue()
volumes =  [{"path": "T1toMNInonlin.nii.gz", "colormap": "blue"},
           {"path": "standard_brain.nii.gz", "colormap": "red"}]
nv_FNIRT.load_volumes(volumes)
nv_FNIRT

In [ ]:
# Visualize results (after registration)
display(Markdown("### Standard MNI (blue) and T1w (red) - after linear registration"))

nv_FNIRT = AnyNiivue()
volumes =  [{"path": "T1toMNIlin.nii.gz", "colormap": "blue"},
           {"path": "standard_brain.nii.gz", "colormap": "red"}]
nv_FNIRT.load_volumes(volumes)
nv_FNIRT

## 3. Combine transformations

Now we will see how we can combine the two registrations performed in the two previous steps.

In this case, this can be used to register the T2w structural image to the MNI standard space, using a single command.

In [ ]:
# Apply sequentially the two transformations estimated above to register the T2w data into the standard MNI space
!applywarp -i T2_brain.nii.gz -r standard_brain --premat=T2toT1_MI.mat -w T1toMNI_warp.nii.gz -o T2toMNI.nii.gz

In [ ]:
# Visualize results (after registration)
display(Markdown("### Standard MNI (blue) and T2w (red) - after registration"))

nv_WARP = AnyNiivue()
volumes =  [{"path": "standard_brain.nii.gz", "colormap": "blue"},
           {"path": "T2toMNI.nii.gz", "colormap": "red"}]
nv_WARP.load_volumes(volumes)
nv_WARP

# **VI. Surface-based Analysis**

Structural MRI studies typically acquire a **T1-weighted** anatomical image - a
high-resolution scan where white matter appears bright, gray matter appears darker, and CSF appears black. Traditional **volume-based analysis** of these images faces a fundamental problem: voxels at cortical folds contain mixed signals from multiple regions and tissue types (the **partial voluming effect**), making precise measurements difficult.

[FreeSurfer](https://surfer.nmr.mgh.harvard.edu/fswiki/FreeSurfer) solves this by reconstructing the cortex as a 2D surface mesh composed of vertices and edges. This surface can be inflated to reveal hidden sulci and allows direct measurement of cortical properties like thickness, curvature, and surface area - avoiding partial voluming effects and providing more anatomically accurate analysis ([link](https://andysbrainbook.readthedocs.io/en/latest/FreeSurfer/FS_ShortCourse/FS_01_BasicTerms.html)).




## Surface Representations

**FreeSurfer** creates several surface representations, each serving a different purpose:

1. **White matter surface**: Inner boundary (gray/white matter interface)
2. **Pial surface**: Outer boundary (gray matter/CSF interface)  
3. **Inflated surface**: Smoothly inflated to reveal sulci (valleys) hidden in folds
4. **Sphere**: Maximally inflated for registration across subjects

Let's visualize these different representations:

In [ ]:
from nilearn import datasets, plotting
import matplotlib.pyplot as plt

# Load the fsaverage template (average surface from ~40 subjects)
fsaverage = datasets.fetch_surf_fsaverage(mesh='fsaverage')

In [ ]:
# White Matter Surface - Inner cortical boundary (gray/white interface)
# Red = gyri (outward), Green = sulci (inward)

view = plotting.view_surf(
    fsaverage['white_left'],
    surf_map=fsaverage['sulc_left'],
    cmap='RdYlGn',
    symmetric_cmap=True,
    title='White Matter Surface',
    black_bg=True
)
view

In [ ]:
# Pial Surface - Description: Outer cortical boundary (gray/CSF interface)
# Red = gyri (outward), Green = sulci (inward)

view = plotting.view_surf(
    fsaverage['pial_left'],
    surf_map=fsaverage['sulc_left'],
    cmap='RdYlGn',
    symmetric_cmap=True,
    title='Pial Surface',
    black_bg=True
)
view

In [ ]:
# Inflated Surface - "Unfolds" the brain, making it easier to see regions that are normally hidden deep in sulci.

view = plotting.view_surf(
    fsaverage['infl_left'],
    surf_map=fsaverage['sulc_left'],
    cmap='RdYlGn',
    symmetric_cmap=True,
    title='Inflated Surface',
    black_bg=True
)
view

## Surface-Based Morphometry (SBM)

Surface-based morphometry involves measuring geometric properties of the cortical surface. Unlike volume-based morphometry (VBM) which looks at tissue density in 3D space, SBM measures properties intrinsic to the cortical sheet.

1. **Sulcal Depth**: Distance from the outer cortical surface to a "mid-cortical" surface. Quantifies how deeply folded each region is.
2. **Curvature**: How much the surface bends at each point. Positive values = concave (sulci/valleys). Negative values = convex (gyri/ridges).
3. **Cortical Thickness**: Distance between white matter and pial surfaces. Typically 2-4mm, varies by region.

Let's visualize these different metrics:

In [ ]:
# Sulcal Depth - Measures cortical folding depth.
# Red = gyri (outward), Green = sulci (inward)

view = plotting.view_surf(
    fsaverage['pial_left'],
    surf_map=fsaverage['sulc_left'],
    cmap='RdYlGn',
    symmetric_cmap=True,
    title='Sulcal Depth',
    black_bg=True
)
view

In [ ]:
#  Curvature - Surface bending
# Red = concave (sulci), Blue = convex (gyri)

view = plotting.view_surf(
    fsaverage['pial_left'],
    surf_map=fsaverage['curv_left'],
    cmap='coolwarm',
    symmetric_cmap=True,
    title='Curvature',
    black_bg=True
)
view

In [ ]:
# Cortical Thickness" - Distance between white and pial surfaces (mm).
# Warmer = thicker cortex

view = plotting.view_surf(
    fsaverage['pial_left'],
    surf_map=fsaverage['thick_left'],
    cmap='hot',
    symmetric_cmap=False,
    title='Cortical Thickness',
    black_bg=True
)
view

## Projecting Functional Data onto Surfaces

One of the key advantages of surface-based analysis is the ability to **visualize functional activation patterns** (from fMRI) on the cortical surface. This reveals activity hidden in deep sulci that would be difficult to see in traditional 3D volume renderings.

Here we take a statistical map from a motor task fMRI study and project it from
volumetric space onto the cortical surface. This process (volume-to-surface mapping) samples the 3D statistical values onto each vertex of the 2D surface mesh.

The result shows motor cortex activation spread across the surface, making it
easier to see the spatial extent along gyri and into sulci:

In [ ]:
# Load example statistical map
motor_images = datasets.fetch_neurovault_motor_task()
stat_map = motor_images.images[0]

# Project volume to surface
from nilearn import surface

texture_left = surface.vol_to_surf(stat_map, fsaverage['pial_left'])

view = plotting.view_surf(
    fsaverage['pial_left'],
    surf_map=texture_left,
    threshold=3.0,
    cmap='cold_hot',
    title='Motor Task Activation on Surface',
    black_bg=True
)
view

# **VII: Final Exercise**

**Objective:** Combine skull-stripping, segmentation, and registration to create a high-confidence White Matter (WM) mask in MNI standard space.

### **Instructions:**
1. **Brain Extraction**: Use `bet` with the `-R` (robust) option on `T1.nii.gz`.
2. **Segmentation**: Use `fast` to segment the brain into different tissues.
3. **Thresholding**: Extract the White Matter (PVE segment 2) and threshold it at **0.9** using `fslmaths` to create a binary mask.
4. **Registration**: Apply the non-linear warp previously calculated during class (`T1toMNI_warp.nii.gz`) to move your **native WM mask** into **MNI standard space** using `applywarp`.
5. **Visualization**: Display the final mask over the standard template.

In [ ]:
# 1. Brain Extraction (Robust)
!bet T1.nii.gz T1_robust_brain.nii.gz -R

In [ ]:
# 2. Tissue Segmentation
!fast -b -B T1_robust_brain.nii.gz

In [ ]:
# 3. Create White Matter Mask (Segment 2)
# We use -thr 0.9 to keep only voxels with >90% probability of being WM
!fslmaths T1_robust_brain_pve_2.nii.gz -thr 0.9 -bin T1_WM_mask_native.nii.gz

In [ ]:
# 4. Apply non-linear registration to standard space
!applywarp -i T1_WM_mask_native.nii.gz -r standard.nii.gz -w T1toMNI_warp.nii.gz -o T1_WM_mask_MNI.nii.gz